In [1]:
import os
import keras

import numpy as np
import cv2
import math
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
(train_x, train_y) , (test_x, test_y) = keras.datasets.cifar10.load_data()

170500096/170498071 [==============================] - 6s 0us/step


In [0]:
x_train_histogram = []
x_test_histogram = []

for i in range(len(train_x)):
  
  chans = cv2.split(train_x[i])
  
  hist_feature = []
  for chan in chans:
    
    hist = cv2.calcHist(chan, [0], None, [16], [0,256] )
    hist_feature.extend(hist.flatten())
  
  x_train_histogram.append(hist_feature)

for i in range(len(test_x)):
  
  chans = cv2.split(test_x[i])
  hist_feature = []
  for chan in chans:
    
    hist = cv2.calcHist(chan, [0] , None, [16], [0,256])
    hist_feature.extend(hist.flatten())
  x_test_histogram.append(hist_feature)
  
  
x_train_histogram = np.asarray(x_train_histogram) 
x_test_histogram = np.asarray(x_test_histogram)


In [0]:
class Hog_Extractor():

    def __init__(self, img, cell_size = 8 , bin_size = 9, gamma = 0.5):

        self.img = img

        if np.ndim(self.img) == 3:
            self.img = cv2.cvtColor(self.img,cv2.COLOR_BGR2GRAY)

        self.cell_size = cell_size
        self.bin_size = bin_size


        self.img = ((img * 1.0) / float(np.max(img))) ** gamma
        self.img = self.img * 255.0

        self.angle_unit = 360 / self.bin_size

        assert type(self.bin_size) == int, "bin size should be integer"
        assert type(self.cell_size) == int, "cell size should be integer"
        assert 360 % self.bin_size == 0, "bin_size should be divisible by 360"

    def extract(self):

        height, width = self.img.shape[0],self.img.shape[1]

        gradient_mag, gradient_ang = self._get_x_y_gradients()
        gradient_mag = abs(gradient_mag)


        cell_gradient_hists = np.zeros((int(height / self.cell_size), int(width / self.cell_size), self.bin_size))
        for i in range(cell_gradient_hists.shape[0]):
            for j in range(cell_gradient_hists.shape[1]):

                cell_mag = gradient_mag[ (i*self.cell_size) : ((i+1)*self.cell_size) , (j*self.cell_size) : ((j+1)*self.cell_size) ]
                cell_ang = gradient_ang[ (i*self.cell_size) : ((i+1)*self.cell_size) , (j*self.cell_size) : ((j+1)*self.cell_size) ]

                cell_gradient_hists[i,j] = self._get_hist(cell_mag, cell_ang)

        hog_vec = []
        for i in range(cell_gradient_hists.shape[0]-1):
            for j in range(cell_gradient_hists.shape[1]-1):

                block_vec = []
                block_vec.extend(cell_gradient_hists[i,j])
                block_vec.extend(cell_gradient_hists[i,j+1])
                block_vec.extend(cell_gradient_hists[i+1,j])
                block_vec.extend(cell_gradient_hists[i+1,j+1])

                l2 = lambda vec : math.sqrt(sum(m ** 2 for m in vec))
                l2_mag = l2(block_vec) + 1e-5

                if l2_mag != 0:
                    normalize = lambda block_vec, l2_mag : [ element / l2_mag for element in block_vec]
                    block_vec = normalize(block_vec, l2_mag)

                hog_vec.append(block_vec)

        return np.asarray(hog_vec)

    def _get_x_y_gradients(self):

        gx = cv2.Sobel(self.img, cv2.CV_64FC1, 1, 0)
        gy = cv2.Sobel(self.img, cv2.CV_64FC1, 0, 1)

        mag, ang = cv2.cartToPolar(gx,gy,angleInDegrees=True)

        return mag, ang

    def _get_hist(self, cell_mag, cell_ang):

        hist_vec = np.zeros(self.bin_size)

        for i in range(cell_mag.shape[0]):
            for j in range(cell_mag.shape[1]):

                mag = cell_mag[i,j]
                ang = cell_ang[i,j]

                lower_ang, upper_ang, weight = self._gradient_interpolation(ang)

                hist_vec[lower_ang] += ( mag * (1-weight) )
                hist_vec[upper_ang] += ( mag * weight )

        return hist_vec


    def _gradient_interpolation(self, angle):

        idx = angle / self.angle_unit
        mod = angle - (self.angle_unit * idx)

        return int(idx % self.bin_size), int((idx+1) % self.bin_size), mod % self.angle_unit


In [0]:
bin_n = 9

def hog(img):
  img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  return Hog_Extractor(img).extract()
x_train_hog = np.asarray([hog(img) for img in train_x])
x_test_hog = np.asarray([hog(img) for img in test_x])

In [0]:
train_y = train_y.astype(int)
test_y = test_y.astype(int)

In [0]:
x_train_hog = np.asarray(x_train_hog, dtype = np.float32)
x_test_hog = np.asarray(x_test_hog, dtype = np.float32)

x_train_hog = x_train_hog.reshape(x_train_hog.shape[0], x_train_hog.shape[1] * x_train_hog.shape[2])
x_test_hog = x_test_hog.reshape(x_test_hog.shape[0], x_test_hog.shape[1] * x_test_hog.shape[2])


In [0]:
SVM_hist = cv2.ml.SVM_create()
SVM_hist.setKernel(cv2.ml.SVM_LINEAR)
SVM_hist.setGamma(5.383)
SVM_hist.setType(cv2.ml.SVM_C_SVC)
SVM_hist.setC(2.67)

#training 
SVM_hist.train(x_train_histogram, cv2.ml.ROW_SAMPLE, train_y)

_, y_hist_train = SVM_hist.predict(x_train_histogram)
_, y_hist_test = SVM_hist.predict(x_test_histogram)

In [0]:
SVM_hog = cv2.ml.SVM_create()
SVM_hog.setKernel(cv2.ml.SVM_LINEAR)
SVM_hog.setGamma(5.383)
SVM_hog.setType(cv2.ml.SVM_C_SVC)
SVM_hog.setC(2.67)

SVM_hog.train(x_train_hog, cv2.ml.ROW_SAMPLE, train_y)

_,y_hog_train = SVM_hog.predict(x_train_hog)
_,y_hog_test = SVM_hog.predict(x_test_hog)

In [41]:
from sklearn.metrics import accuracy_score

train_hist_acc = accuracy_score(train_y, y_hist_train)
test_hist_acc = accuracy_score(test_y, y_hist_test)

print(f' hist train acc : {train_hist_acc*100}%')
print(f' hist test acc : {test_hist_acc*100}%')

 hist train acc : 9.388%
 hist test acc : 9.55%


In [42]:
train_hog_acc = accuracy_score(train_y, y_hog_train)
test_hog_acc = accuracy_score(test_y, y_hog_test)

print(f' hog train acc : {train_hog_acc*100}%')
print(f' hog test acc : {test_hog_acc*100}%')

 hog train acc : 22.664%
 hog test acc : 22.63%
